## HERE FREE ACCOUNT

https://developer.here.com/pricing

Free Transactions per month 250K

Pay per additional Transactions 1000 / $1

In [4]:
# !pip install here_location_services

In [3]:
import json
import os
from datetime import datetime
from here_location_services import LS
from here_location_services.config.routing_config import ROUTING_RETURN

In [18]:
import here_location_services as hls

In [19]:
hls.__version__

'0.3.0'

In [4]:
LS_API_KEY = os.environ.get("HERE_API_KEY")  # Get API KEY from environment.
ls = LS(api_key=LS_API_KEY)

### HERE geocoding

In [5]:
address = "Monash University, Clayton, Vic"
geo = ls.geocode(query=address)
geo

In [6]:
print(geo)

{"items": [{"access": [{"lat": -37.91315, "lng": 145.13562}], "address": {"city": "Melbourne", "countryCode": "AUS", "countryName": "Australia", "district": "Clayton", "label": "Monash University, Scenic Blvd, Clayton VIC 3168, Australia", "postalCode": "3168", "state": "Victoria", "stateCode": "VIC", "street": "Scenic Blvd"}, "categories": [{"id": "800-8200-0173", "name": "Higher Education", "primary": true}], "id": "here:pds:place:036r1r0q-860725be724e464aab14f806f8d8ab55", "position": {"lat": -37.91319, "lng": 145.13569}, "resultType": "place", "scoring": {"fieldScore": {"district": 1.0, "placeName": 1.0, "state": 1.0}, "queryScore": 1.0}, "title": "Monash University"}, {"access": [{"lat": -37.91322, "lng": 145.13556}], "address": {"city": "Melbourne", "countryCode": "AUS", "countryName": "Australia", "district": "Clayton", "label": "Monash University, Scenic Blvd, Clayton VIC 3168, Australia", "postalCode": "3168", "state": "Victoria", "stateCode": "VIC", "street": "Scenic Blvd"}, 

### HERE routing APIs: Car Driving

Respone data type: https://developer.here.com/documentation/routing/dev_guide/topics/resource-type-route-summary.html

In [7]:
orig = [-37.89577598106494, 145.09038583978858]
dest = [-37.91291561061539, 145.13363175213212]

In [8]:
result = ls.car_route(
    origin=orig,
    destination=dest,
    return_results=[
#         ROUTING_RETURN.polyline,
#         ROUTING_RETURN.elevation,
#         ROUTING_RETURN.instructions,
#         ROUTING_RETURN.actions,
        ROUTING_RETURN.summary
    ],
)

In [12]:
print(result)

{"routes": [{"id": "68e6c315-1b6f-49d3-90f5-841bd4e6fef5", "sections": [{"arrival": {"place": {"location": {"lat": -37.91358, "lng": 145.13297}, "originalLocation": {"lat": -37.9129156, "lng": 145.1336318}, "type": "place"}, "time": "2021-08-06T09:58:20+10:00"}, "departure": {"place": {"location": {"lat": -37.8957648, "lng": 145.0902832}, "originalLocation": {"lat": -37.8957761, "lng": 145.0903858}, "type": "place"}, "time": "2021-08-06T09:49:38+10:00"}, "id": "c797db8b-95fd-4544-b96d-937d40e8247e", "summary": {"baseDuration": 430, "duration": 522, "length": 5726}, "transport": {"mode": "car"}, "type": "vehicle"}]}]}


In [13]:
# Using trip start_time
start_time = datetime.strptime("27/05/2012 23:30",'%d/%m/%Y %H:%M')
result = ls.car_route(
    origin=orig,
    destination=dest,
    departure_time=start_time,
    return_results=[ ROUTING_RETURN.summary ],
)

In [21]:
RoutingResponse??

Object `RoutingResponse` not found.


In [22]:
result??

Type:        RoutingResponse
String form: {"routes": [{"id": "57385037-f3fa-42cb-9035-6d6b0025c587", "sections": [{"arrival": {"place": {"location": {"lat": -37.91358, "lng": 145.13297}, "originalLocation": {"lat": -37.9129156, "lng": 145.1336318}, "type": "place"}, "time": "2012-05-27T23:37:16+10:00"}, "departure": {"place": {"location": {"lat": -37.8957648, "lng": 145.0902832}, "originalLocation": {"lat": -37.8957761, "lng": 145.0903858}, "type": "place"}, "time": "2012-05-27T23:30:00+10:00"}, "id": "06f8b572-9629-494b-b5fc-4ae2d0accf6e", "summary": {"baseDuration": 436, "duration": 436, "length": 5289}, "transport": {"mode": "car"}, "type": "vehicle"}]}]}
File:        ~/.pyenv/versions/3.8.5/envs/.ml/lib/python3.8/site-packages/here_location_services/responses.py
Source:     
class RoutingResponse(ApiResponse):
    """A class representing the search routing API response data."""

    def __init__(self, **kwargs):
        super().__init__()
        self._filters = {"routes": None}
  

In [14]:
print(result)

{"routes": [{"id": "57385037-f3fa-42cb-9035-6d6b0025c587", "sections": [{"arrival": {"place": {"location": {"lat": -37.91358, "lng": 145.13297}, "originalLocation": {"lat": -37.9129156, "lng": 145.1336318}, "type": "place"}, "time": "2012-05-27T23:37:16+10:00"}, "departure": {"place": {"location": {"lat": -37.8957648, "lng": 145.0902832}, "originalLocation": {"lat": -37.8957761, "lng": 145.0903858}, "type": "place"}, "time": "2012-05-27T23:30:00+10:00"}, "id": "06f8b572-9629-494b-b5fc-4ae2d0accf6e", "summary": {"baseDuration": 436, "duration": 436, "length": 5289}, "transport": {"mode": "car"}, "type": "vehicle"}]}]}


In [15]:
def get_info(los_res):
    """
    This function parse result from HERE route api, and return travel_time and trip length
    Parameters
        los_res: result from HERE rout api
    Return
        (trip_duration, trip_length)
    """
    res = los_res.routes[0]['sections'][0]['summary']
    return res['duration'], res['length']

def collect_here_los(ls, orig, dest, departure_time=None):
    """
    This function collect level of service (travel time) from HERE APIs
    This return duration in seconds, and len in meter.
    Parameters
        ls: HERE location services (LS) object
        orig: origin location as a [lattitue, longtitude] list
        dest: destination location as a [lattitue, longtitude] list
        departure_time: the trip's departure time in datetime format
    Return
        [car_duration, car_length, cycle_duration, cycle_length, walk_duration, walk_length]
    """    
    # Get car duration and length
    car_res = ls.car_route(origin=orig, destination=dest, departure_time=departure_time, 
                        return_results=[ROUTING_RETURN.summary])
    car_dur, car_len = get_info(car_res)
    # Get cycle duration and length
    cycle_res = ls.bicycle_route(origin=orig, destination=dest, departure_time=departure_time, 
                        return_results=[ROUTING_RETURN.summary])
    cycle_dur, cycle_len = get_info(cycle_res)
    # Get walk duration and length
    walk_res = ls.pedestrian_route(origin=orig, destination=dest, departure_time=departure_time, 
                        return_results=[ROUTING_RETURN.summary])
    walk_dur, walk_len = get_info(walk_res)
    return [car_dur, car_len, cycle_dur, cycle_len, walk_dur, walk_len]    

In [16]:
get_info(result)

(436, 5289)

In [17]:
collect_here_los(ls, orig, dest, start_time)

[436, 5289, 1372, 5110, 5088, 5032]